## 1. Importing Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("https://github.com/statisticspoland/ecoicop_classification/blob/master/products_allshops_dataset.xlsx?raw=true")

In [3]:
df

,produkt,kategoria
0,"""słynne roślinne""",Margaryna i inne tłuszcze roślinne
1,#Hejki - Emotki lizaki ręcznie robione o smaka...,Wyroby cukiernicze
2,100% Pur jus d'orange - sok pomarańczowy z mią...,Soki owocowe i warzywne
3,100% sukraloza bez cukru (substancje słodzące),Sztuczne substytuty cukru
4,100% z brzoskwiń produkt owocowy słodzony zag....,"Dżemy, marmolady i miód"
...,...,...
16625,Żywiec Zdrój ze smakiem truskawki Napój niegaz...,Wody mineralne lub źródlane
16626,Żywiec green tea&gruszka,Napoje orzeźwiające
16627,Żywioł - Woda źródlana delikatnie gazowana,Wody mineralne lub źródlane
16628,Żywioł - Woda źródlana gazowana,Wody mineralne lub źródlane


In [4]:
#We drop categories with few rows (less than 50) to streamline the analysis:
df = df[df.groupby("kategoria")["kategoria"].transform("count").ge(50)]
df = df.reset_index(drop=True)

In [5]:
#Set image renderer to display the images in nbviewer
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## 2. Generating drift in data with NMF

NMF: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

In [6]:
#First we do a bag of words transformation for text feature "produkt"
#This is used both by the model and drift detecters
#Vectorizer and model parameters taken from https://github.com/statisticspoland/ecoicop_classification/blob/master/Random_Forest/random_forest_results.py
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF

#We reduce max features to make the analysis a little faster. For a better model performance the parameter can just be removed
vectorizer = CountVectorizer(token_pattern='\w\w+|[1-9]\.[1-9]\%|[1-9]\,[1-9]\%|[1-9]\.[1-9]|[1-9]\,[1-9]|[1-9]\%',  max_features=500)
vectorizer.fit(df['produkt'])

X_vectorized = pd.DataFrame(vectorizer.transform(df["produkt"]).todense(), columns=vectorizer.get_feature_names())
X_vectorized["kategoria"] = df["kategoria"]

print("From text column: ")
display(df[0:5])
print("To bag of words: ")
display(X_vectorized)

From text column: 


,produkt,kategoria
0,"""słynne roślinne""",Margaryna i inne tłuszcze roślinne
1,#Hejki - Emotki lizaki ręcznie robione o smaka...,Wyroby cukiernicze
2,100% Pur jus d'orange - sok pomarańczowy z mią...,Soki owocowe i warzywne
3,100% z brzoskwiń produkt owocowy słodzony zag....,"Dżemy, marmolady i miód"
4,100% z czarnych porzeczek produkt owocowy słod...,"Dżemy, marmolady i miód"


To bag of words: 


,"1,5",10,100,1000,110,12,120,125,130,135,...,świeże,źródlana,żelki,żucia,żurawina,żurawiną,żurek,żywiec,żółty,kategoria
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Margaryna i inne tłuszcze roślinne
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Wyroby cukiernicze
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Soki owocowe i warzywne
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Dżemy, marmolady i miód"
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Dżemy, marmolady i miód"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16258,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Wody mineralne lub źródlane
16259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Napoje orzeźwiające
16260,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Wody mineralne lub źródlane
16261,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Wody mineralne lub źródlane


In [7]:
#We try to split the data to two "categories"
nmf = NMF(n_components=2, init="random")
W = nmf.fit_transform(X_vectorized.drop(columns=["kategoria"]))

In [8]:
import numpy as np
sort_idx = np.argsort(W[:, 0])

In [9]:
#import plotly.express as px
#fig = px.line(W[:, 0][sort_idx][::-1], width=1000)
#fig.show(renderer="svg")

In [10]:
X_vectorized = X_vectorized.iloc[sort_idx[::-1]]

In [11]:
X_vectorized

,"1,5",10,100,1000,110,12,120,125,130,135,...,świeże,źródlana,żelki,żucia,żurawina,żurawiną,żurek,żywiec,żółty,kategoria
10446,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Napoje orzeźwiające
943,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Napoje orzeźwiające
941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Napoje orzeźwiające
10456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Napoje orzeźwiające
10457,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Napoje orzeźwiające
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Dżemy, marmolady i miód"
3267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Dżemy, marmolady i miód"
3266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Dżemy, marmolady i miód"
6349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Wędliny


In [12]:
#Also order original dataframe:
df = df.iloc[sort_idx[::-1]]

In [13]:
#TODO VISUALIZE NMF
#TODO ENSURE ALL CATEGORIES IN DRIFT?

## 3. Model lifecycle and drift detection

In [14]:
#To illustrate drift detection we simulate a model lifecycle and drift detection in various phases

In [15]:
#Splitting data:
start_data = df[100:].copy().reset_index(drop=True)
new_data_data_drift = df[0:100].copy().reset_index(drop=True)

In [16]:
#TODO: NEW DATA NON DRIFT
#TODO: NEW DATA CONCEPT DRIFT

### 3.1 Training a model with training data

In [17]:
#We have some training data, so let's train a model

In [18]:
#Train test split
from sklearn.model_selection import train_test_split
train, test = train_test_split(start_data, test_size=0.2, stratify=start_data["kategoria"])

In [19]:
#Preprocessing
#Bag of words transformation
vectorizer = CountVectorizer(token_pattern='\w\w+|[1-9]\.[1-9]\%|[1-9]\,[1-9]\%|[1-9]\.[1-9]|[1-9]\,[1-9]|[1-9]\%',  max_features=500)
train_x = vectorizer.fit_transform(train["produkt"])

In [20]:
#BUILD MODEL
#Model taken from https://github.com/statisticspoland/ecoicop_classification/blob/master/Random_Forest/random_forest_results.py

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
    n_estimators=200,
    criterion='gini',
    min_samples_leaf=1,
    min_samples_split=3,
    max_features='log2',
    bootstrap=False,
    oob_score=False,
    warm_start=False,
    class_weight=None
)

clf.fit(train_x, train["kategoria"])

RandomForestClassifier(bootstrap=False, max_features='log2',
                       min_samples_split=3, n_estimators=200)

In [21]:
test_x = vectorizer.transform(test["produkt"])
test_predictions = clf.predict(test_x)

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
print("Testing model accuracy for sanity")
test_accuracy = accuracy_score(test["kategoria"], test_predictions)
print(f"Model accuracy: {test_accuracy}")
print("Model accuracy seems to be fine for our purposes")

Testing model accuracy for sanity
Model accuracy: 0.8085369625734612
Model accuracy seems to be fine for our purposes


### 3.2 Predicting on new data and detecting data drift

In [24]:
#Building reference data:
#We use the data that the model was trained on:
reference_data =  pd.DataFrame(vectorizer.transform(train["produkt"]).todense(), columns=vectorizer.get_feature_names())
#Get X Columns:
x_cols = list(reference_data.columns.values)
#Add Y Column
reference_data["kategoria"] = train["kategoria"].reset_index(drop=True)
reference_data["prediction"] = clf.predict(vectorizer.transform(train["produkt"]))

In [25]:
#Preprocess new incoming data

In [26]:
new_data_data_drift_vec = pd.DataFrame(vectorizer.transform(new_data_data_drift["produkt"]).todense(), columns=vectorizer.get_feature_names())

In [27]:
#Let's check the drift with KS
from alibi_detect.cd import KSDrift
ks_detector = KSDrift(reference_data[x_cols].to_numpy(), p_val=0.05, correction="bonferroni")

In [28]:
preds = ks_detector.predict(new_data_data_drift_vec.to_numpy(), drift_type="batch", return_p_val=True, return_distance=True)

In [29]:
print(f"KS Drift detected? {'Yes' if preds['data']['is_drift'] else 'No'}")

KS Drift detected? Yes


In [30]:
p_values = preds["data"]["p_val"]
p_idx = np.argsort(p_values)
p_values = p_values[p_idx]

In [31]:
ks_drift_data = pd.DataFrame(data={"Feature": np.array(x_cols)[p_idx][0:15], "Distance": preds["data"]["distance"][p_idx][0:15], "P-Value": p_values[0:15]})
ks_drift_data["Drift?"] = ks_drift_data["P-Value"] < 0.05
print("Alibi Detect KSDrift for 15 most drifted features")
ks_drift_data

Alibi Detect KSDrift for 15 most drifted features


,Feature,Distance,P-Value,Drift?
0,napój,0.939938,0.000000e+00,True
1,gazowany,0.722034,0.000000e+00,True
2,ml,0.685661,0.000000e+00,True
3,smaku,0.601199,9.195117e-35,True
4,500,0.322390,1.174557e-09,True
5,250,0.188964,1.437088e-03,True
6,niegazowany,0.113813,1.423196e-01,False
7,black,0.097448,2.851609e-01,False
8,cytrynowym,0.087912,4.052379e-01,False
9,330,0.045282,9.817010e-01,False


In [32]:
#TODO: DISTANCE LINE
from plotly import express as px
fig = px.scatter({"p-value": p_values[0:15], "feature": np.array(x_cols)[p_idx][0:15]}, x="feature", y="p-value", range_y=[0,1],
                 title="P-Values for a sample of features")
fig.add_hline(y=0.05, line_color="red")
fig.show()

In [33]:
from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab, CatTargetDriftTab

In [34]:
drift_dashboard = Dashboard(tabs=[DataDriftTab])

In [35]:
drift_dashboard.calculate(reference_data[x_cols], new_data_data_drift_vec.copy())

In [36]:
drift_dashboard.save("data_dift_report.html")

In [37]:
#TODO: COMPARISON: EVIDENTLY, ALIBI DETECT, high treshold for evidently

In [38]:
#Testing with predicted categories
test_predictions = clf.predict(vectorizer.transform(new_data_data_drift["produkt"]))

In [39]:
new_data_data_drift_vec["prediction"] = test_predictions

In [40]:
cat_target_drift_dashboard = Dashboard(tabs=[CatTargetDriftTab])
column_mapping = {}
column_mapping["prediction"] = "prediction"
column_mapping["numerical_features"] = x_cols

In [41]:
cat_target_drift_dashboard.calculate(reference_data.drop(columns=["kategoria"]), new_data_data_drift_vec.copy(), column_mapping=column_mapping)

d:\git\drift-detection-simulation\env\lib\site-packages\scipy\stats\stats.py:6125: RuntimeWarning:

divide by zero encountered in true_divide



In [42]:
cat_target_drift_dashboard.save("predicted_target_drift.html")

In [43]:
#TODO: WHY THIS IS NOT CONCEPT DRIFT

### 3.3 Real Y values and model performance

In [44]:
new_data_data_drift_vec["kategoria"] = new_data_data_drift["kategoria"] #Let's assume we get real "kategoria" values

In [45]:
new_accuracy = accuracy_score(new_data_data_drift_vec["kategoria"], new_data_data_drift_vec["prediction"])

In [46]:
#Accuracy seems to even increase. Why? Because concept drift has not occured
new_accuracy

0.89

In [47]:
from evidently.tabs import ClassificationPerformanceTab

In [48]:
classification_performance_dashboard = Dashboard(tabs=[ClassificationPerformanceTab])

In [49]:
column_mapping["target"] = "kategoria"

In [50]:
#sample to speed up analysis
sample_reference, _ = train_test_split(reference_data, train_size=100, stratify=reference_data["kategoria"])

In [51]:
#Sample features to speed up report
from random import sample
x_cols_sample = sample(x_cols, 10)
column_mapping["numerical_features"] = x_cols_sample

In [52]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    classification_performance_dashboard.calculate(
        reference_data=sample_reference[x_cols_sample+["kategoria", "prediction"]].copy(),
        current_data=new_data_data_drift_vec[x_cols_sample+["kategoria", "prediction"]].copy(),
        column_mapping=column_mapping
    )

In [53]:
classification_performance_dashboard.save("classification_performance.html")

## 4. Other temp

In [54]:
#Other alibi_detect algorithm:
from alibi_detect.cd import MMDDrift
mmm_detector= MMDDrift(reference_data[x_cols].to_numpy(), p_val=0.05, backend='pytorch')

No GPU detected, fall back on CPU.


In [57]:
preds_mmm = mmm_detector.predict(new_data_data_drift_vec[x_cols].to_numpy(), return_p_val=True, return_distance=True)

In [58]:
preds_mmm

{'data': {'is_drift': 1,
  'distance': 0.34631967544555664,
  'p_val': 0.0,
  'threshold': 0.05,
  'distance_threshold': 0.0009915829},
 'meta': {'name': 'MMDDriftTorch',
  'detector_type': 'offline',
  'data_type': None,
  'backend': 'pytorch'}}